# 분류 분석

## 이진 고객 이탈

고객의 웹사이트 광고를 제공하는 마케팅 대행사에서 당사의 고객이탈율이 꽤 높다는 사실을 알게됐습니다.<br>
그 회사에서는 즉시 고객 관리자들을 할당했지만, 어떤 고객이 이탈 할 것인지 예측하는 기계학습 모델을 만들어서 가장 이탈확률이 높은 고객에게 우선적으로 고객 관리자를 배치할 수 있기를 원합니다.<br>
고객이 잠재적 이탈 고객인지 여부를 분류하는 분류모델을 만드세요.

데이터는 customer_churn.csv로 저장돼있습니다. 아래는 각 컬럼의 정의입니다.
- Name : 회사의 최근 담당자의 이름
- Age : 고객의 나이
- Total_Purchase : 구매한 총 광고
- Account_Manager : 바이너리 0 = 고객 관리자 없음, 1 = 고객 관리자 할당됨
- Years : 거래 유지 연도수
- Num_sites : 서비스를 이용 중인 웹사이트 수
- Onboard_date : 마지막 연락처가 등록된 날짜
- Location : 고객 사무실 주소
- Company : 고객 회사의 이름

모델 생성 후에는 고객이 제공하는 new_customers.csv 파일에 저장된 새로운 데이터에 대한 예측값을 제시하세요. <br>
고객은 이 데이터를 통해 관리가 필요한 고객들을 알고 싶어 합니다.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.preprocessing import RobustScaler # 스케일러다! (x-중앙값)/IQR => 이상치에 영향을 덜 받게 하려고 씀 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

### 데이터 불러오기

In [2]:
# churn 1 : 이탈함 아니면 이탈 안함
# new_customers.csv 이거 테스트로 합시다~~~

In [3]:
df = pd.read_csv("./data/customer_churn.csv")
df.head()

,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company,Churn
0,Cameron Williams,42.0,11066.80,0,7.22,8.0,2013-08-30 07:00:40,"10265 Elizabeth Mission Barkerburgh, AK 89518",Harvey LLC,1
1,Kevin Mueller,41.0,11916.22,0,6.50,11.0,2013-08-13 00:38:46,"6157 Frank Gardens Suite 019 Carloshaven, RI 1...",Wilson PLC,1
2,Eric Lozano,38.0,12884.75,0,6.67,12.0,2016-06-29 06:20:07,"1331 Keith Court Alyssahaven, DE 90114","Miller, Johnson and Wallace",1
3,Phillip White,42.0,8010.76,0,6.71,10.0,2014-04-22 12:43:12,"13120 Daniel Mount Angelabury, WY 30645-4695",Smith Inc,1
4,Cynthia Norton,37.0,9191.58,0,5.56,9.0,2016-01-19 15:31:15,"765 Tricia Row Karenshire, MH 71730",Love-Jones,1


### 데이터 확인

In [4]:
df.shape

(900, 10)

In [5]:
df.dtypes

Names               object
Age                float64
Total_Purchase     float64
Account_Manager      int64
Years              float64
Num_Sites          float64
Onboard_date        object
Location            object
Company             object
Churn                int64
dtype: object

In [6]:
df.isna().sum()

Names              0
Age                0
Total_Purchase     0
Account_Manager    0
Years              0
Num_Sites          0
Onboard_date       0
Location           0
Company            0
Churn              0
dtype: int64

### 머신러닝용 데이터 전처리

우리는 수치형 데이터만 이용할 예정입니다. Account_Manager는 다루기 어렵지 않기 때문에 모델학습에 포함할 수도 있겠지만 무작위로 할당된 것이기 때문에 큰 의미가 없을 수 있습니다.

In [7]:
df = df.drop(["Names","Onboard_date","Location","Company"], axis = 1)

In [8]:
df.head()

,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn
0,42.0,11066.80,0,7.22,8.0,1
1,41.0,11916.22,0,6.50,11.0,1
2,38.0,12884.75,0,6.67,12.0,1
3,42.0,8010.76,0,6.71,10.0,1
4,37.0,9191.58,0,5.56,9.0,1


In [9]:
df["Churn"].unique()

array([1, 0])

# 일단 독립변수 종속변수 나눌것임

In [10]:
x = df.drop(["Churn"], axis = 1)
y = df["Churn"]

# 언더 스코어 적용하기 전

In [11]:
x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size = 0.3,
    stratify = y,
    random_state = 666
)

In [12]:
len(x_train), len(x_test)

(630, 270)

In [13]:
# 모델에 넣어야지!
logi = LogisticRegression()
logi.fit(x_train, y_train)

pred = logi.predict(x_test)

obunlu = confusion_matrix(y_test, pred)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
obunlu

array([[222,   3],
       [ 37,   8]])

In [15]:
print(accuracy_score(y_test, pred))

0.8518518518518519


In [46]:
model = sm.Logit(y_train, x_train)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.404005
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  Churn   No. Observations:                  630
Model:                          Logit   Df Residuals:                      626
Method:                           MLE   Df Model:                            3
Date:                Fri, 21 Nov 2025   Pseudo R-squ.:                  0.1033
Time:                        09:00:32   Log-Likelihood:                -254.52
converged:                       True   LL-Null:                       -283.85
Covariance Type:            nonrobust   LLR p-value:                 1.136e-12
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Age               -0.0971      0.015     -6.310      0.000      -0.127      -0.067
Total_Purchase    -0.0002   4.45e-05     -4.299      0.000      -0.000      -0.000
Years              0.0106      0.083      0.127      0.899      -0.153       0.174
Num_Sites          0.4799      0.063      7.576      0.000       0.356       0.604
==================================================================================
"""

In [47]:
# 모델에 넣어야지!
logi = LogisticRegression()
logi.fit(x_train, y_train)

pred = logi.predict(x_test)

obunlu = confusion_matrix(y_test, pred)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [48]:
obunlu

array([[212,  13],
       [ 18,  27]])

In [49]:
print(accuracy_score(y_test, pred))

0.8851851851851852


# p_value 나가리 된 독립변수 제외 : Years 제거

In [45]:
x_train, x_test, y_train, y_test = train_test_split(
    x.drop(["Account_Manager"],axis=1),
    y,
    test_size = 0.3,
    stratify = y,
    random_state = 666
)

In [ ]:
model = sm.Logit(y_train, x_train)
results = model.fit()
results.summary()

In [ ]:
# 모델에 넣어야지!
logi = LogisticRegression()
logi.fit(x_train, y_train)

pred = logi.predict(x_test)

obunlu = confusion_matrix(y_test, pred)

# 언더 스코어 적용한 후

In [21]:
# Churn 비율 어떻게 됨?
df["Churn"].value_counts()

# 아 언더 스코어를 해야하겠구나... 예측모델이 뻑나겠구나~~~~
# 하는것을 알 았 습 니 다

Churn
0    750
1    150
Name: count, dtype: int64

In [22]:
# 언더 스코어 적용하기 위해서 150으로 할거임~~
stay_index = df[df["Churn"] == 0].sample(150, random_state = 666).index.tolist() 
out_index = df[df["Churn"] == 1].index.tolist() # 남을겁니다. # 이탈잡니다ㅇㅇ

In [23]:
# 반반 뽑아야지~~~
random_index = stay_index + out_index

In [24]:
stay_index[0]
x.iloc[0]

Age                   42.00
Total_Purchase     11066.80
Account_Manager        0.00
Years                  7.22
Num_Sites              8.00
Name: 0, dtype: float64

In [25]:
x.head()

,Age,Total_Purchase,Account_Manager,Years,Num_Sites
0,42.0,11066.80,0,7.22,8.0
1,41.0,11916.22,0,6.50,11.0
2,38.0,12884.75,0,6.67,12.0
3,42.0,8010.76,0,6.71,10.0
4,37.0,9191.58,0,5.56,9.0


In [26]:
sample_x = x.iloc[random_index]
sample_y = df["Churn"][random_index]

In [27]:
sample_y.count()

np.int64(300)

In [28]:
len(sample_x), sample_y.count()

(300, np.int64(300))

In [29]:
x_train, x_test, y_train, y_test = train_test_split(
    sample_x,
    sample_y,
    test_size = 0.3,
    stratify = sample_y,
    random_state = 666
)

# 모델 평가 2

In [30]:
# 모델에 넣어야지!
logi2 = LogisticRegression()
logi2.fit(x_train, y_train)

pred = logi2.predict(x_test)

obunlu = confusion_matrix(y_test, pred)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [31]:
obunlu

array([[38,  7],
       [ 8, 37]])

In [32]:
print(accuracy_score(y_test, pred))

0.8333333333333334


In [33]:
# 애들일단 리포트 찍어봐 미쳐부리겠네

In [34]:
model = sm.Logit(y_train, x_train)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.570476
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  Churn   No. Observations:                  210
Model:                          Logit   Df Residuals:                      205
Method:                           MLE   Df Model:                            4
Date:                Fri, 21 Nov 2025   Pseudo R-squ.:                  0.1770
Time:                        08:56:10   Log-Likelihood:                -119.80
converged:                       True   LL-Null:                       -145.56
Covariance Type:            nonrobust   LLR p-value:                 1.736e-10
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Age                -0.0872      0.022     -4.006      0.000      -0.130      -0.045
Total_Purchase     -0.0001      6e-05     -2.355      0.019      -0.000   -2.37e-05
Account_Manager     0.0621      0.315      0.197      0.844      -0.555       0.680
Years               0.0265      0.118      0.224      0.823      -0.205       0.258
Num_Sites           0.5329      0.093      5.732      0.000       0.351       0.715
===================================================================================
"""

In [40]:
# Total_Purchase 랑 Account_Manager 상관관계가 나가리입니다~
x_train, x_test, y_train, y_test = train_test_split(
    sample_x.drop(["Account_Manager"],axis=1),
    sample_y,
    test_size = 0.3,
    stratify = sample_y,
    random_state = 666
)

In [41]:
# 모델에 넣어야지!
logi3 = LogisticRegression()
logi3.fit(x_train, y_train)

pred = logi3.predict(x_test)

obunlu = confusion_matrix(y_test, pred)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [42]:
obunlu

array([[38,  7],
       [ 6, 39]])

In [43]:
model = sm.Logit(y_train, x_train)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.570568
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  Churn   No. Observations:                  210
Model:                          Logit   Df Residuals:                      206
Method:                           MLE   Df Model:                            3
Date:                Fri, 21 Nov 2025   Pseudo R-squ.:                  0.1768
Time:                        08:58:50   Log-Likelihood:                -119.82
converged:                       True   LL-Null:                       -145.56
Covariance Type:            nonrobust   LLR p-value:                 3.860e-11
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Age               -0.0869      0.022     -4.003      0.000      -0.129      -0.044
Total_Purchase    -0.0001   5.99e-05     -2.346      0.019      -0.000   -2.31e-05
Years              0.0257      0.118      0.218      0.828      -0.206       0.257
Num_Sites          0.5347      0.093      5.777      0.000       0.353       0.716
==================================================================================
"""

In [44]:
print(accuracy_score(y_test, pred))

0.8555555555555555


In [64]:
ndf = pd.read_csv("./data/new_customers.csv", index_col = False)
ndf.head() # 오잉? 컬럼 밀렸네

,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company
0,Andrew Mccall,37.0,9935.53,1,7.71,8.0,2011-08-29 18:37:54,"38612 Johnny Stravenue Nataliebury, WI 15717-8316",King Ltd
1,Michele Wright,23.0,7526.94,1,9.28,15.0,2013-07-22 18:19:54,"21083 Nicole Junction Suite 332, Youngport, ME...",Cannon-Benson
2,Jeremy Chang,65.0,100.00,1,1.00,15.0,2006-12-11 07:48:13,"085 Austin Views Lake Julialand, WY 63726-4298",Barron-Robertson
3,Megan Ferguson,32.0,6487.50,0,9.40,14.0,2016-10-28 05:32:13,"922 Wright Branch North Cynthialand, NC 64721",Sexton-Golden
4,Taylor Young,32.0,13147.71,1,10.00,8.0,2012-03-20 00:36:46,Unit 0789 Box 0734 DPO AP 39702,Wood LLC


In [52]:
ndf = ndf.drop(["Names","Onboard_date","Location","Company"], axis = 1)

In [65]:
# 학습했던 스케일링 데이터
# 테스트 데이터도 똑같이 스케일링 해야함!

In [67]:
ndf.head()

,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company
0,Andrew Mccall,37.0,9935.53,1,7.71,8.0,2011-08-29 18:37:54,"38612 Johnny Stravenue Nataliebury, WI 15717-8316",King Ltd
1,Michele Wright,23.0,7526.94,1,9.28,15.0,2013-07-22 18:19:54,"21083 Nicole Junction Suite 332, Youngport, ME...",Cannon-Benson
2,Jeremy Chang,65.0,100.00,1,1.00,15.0,2006-12-11 07:48:13,"085 Austin Views Lake Julialand, WY 63726-4298",Barron-Robertson
3,Megan Ferguson,32.0,6487.50,0,9.40,14.0,2016-10-28 05:32:13,"922 Wright Branch North Cynthialand, NC 64721",Sexton-Golden
4,Taylor Young,32.0,13147.71,1,10.00,8.0,2012-03-20 00:36:46,Unit 0789 Box 0734 DPO AP 39702,Wood LLC


In [69]:
# 모델에 넣어야지!
pred_result = logi3.predict(ndf)

pred_result

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Account_Manager
- Company
- Location
- Names
- Onboard_date
